# IRCADb Segmentation

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#IRCADb-Segmentation" data-toc-modified-id="IRCADb-Segmentation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>IRCADb Segmentation</a></span><ul class="toc-item"><li><span><a href="#Resources" data-toc-modified-id="Resources-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Resources</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Code" data-toc-modified-id="Code-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Code</a></span></li></ul></li></ul></div>

## Resources
* Article: 
    * https://medium.com/@fabio.sancinetti/u-net-convnet-for-ct-scan-segmentation-6cc0d465eed3
* Dataset:
    * https://www.ircad.fr/research/3d-ircadb-01/

## Imports

In [4]:
# Imports jtplot submodule from jupyterthemes, sets notebook theme for plots.
# Nice to have if you are using jupyterthemes, otherwise ignore this. 
try:
    __import__('imp').find_module('jupyterthemes')
    from jupyterthemes import jtplot
    jtplot.style()
except ImportError:
    pass

# General imports
import pydicom
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline

# Machine learning imports
import tensorflow as tf
import keras
from keras import backend as K
from keras import losses
from keras.models import Model, Sequential
from keras.objectives import categorical_crossentropy
from keras.layers import Input, concatenate
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, Conv2DTranspose, ZeroPadding2D
from keras.optimizers import Adam, Adadelta, Adamax, Nadam, Adagrad, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback
from keras.initializers import RandomUniform, RandomNormal

ModuleNotFoundError: No module named 'tensorflow'

## Code

In [3]:
# Load in the DICOM files
DICOM_DATA = 'data/'